In [ ]:
pip install numpy pandas matplotlib seaborn scikit-learn

In [4]:
import pandas as pd

# Update file path as per your system
dataset_path = "dataset1.csv"
df = pd.read_csv(dataset_path, skiprows=4)
print("Dataset loaded successfully.")

print(df.head())




Dataset loaded successfully.
   # Columns: time  avg_rss12  var_rss12  avg_rss13  var_rss13  avg_rss23  \
0                0      35.00       3.67      16.50       3.77      14.00   
1              250      28.50       3.35      17.50       3.77      12.25   
2              500      35.50       2.87      15.75       2.86      17.75   
3              750      29.75      12.19      16.25       2.17      20.75   
4             1000      27.00       2.12      16.75       0.83      24.75   

   var_rss23  
0       1.63  
1       3.90  
2       5.07  
3       2.59  
4       1.64  


In [ ]:
import time

# Function to simulate streaming data
def simulate_streaming_data(data, delay=0.5):
    """
    Simulates streaming data by yielding one row at a time.
    Parameters:
        data (pd.DataFrame): The input dataset.
        delay (float): Time delay (in seconds) between each row emission.
    """
    for index, row in data.iterrows():
        print(f"\nStreaming row {index + 1} of {len(data)}:")
        print(row.to_dict())  # Display the row data as a dictionary
        yield row
        time.sleep(delay)  # Simulate delay between streams

# Simulate streaming the dataset
print("\nSimulating data stream...")
for streamed_row in simulate_streaming_data(df, delay=0.5):
    # Process the streamed row here (e.g., perform analysis or pass to a model)
    pass



Simulating data stream...

Streaming row 1 of 480:
{'# Columns: time': 0.0, 'avg_rss12': 35.0, 'var_rss12': 3.67, 'avg_rss13': 16.5, 'var_rss13': 3.77, 'avg_rss23': 14.0, 'var_rss23': 1.63}

Streaming row 2 of 480:
{'# Columns: time': 250.0, 'avg_rss12': 28.5, 'var_rss12': 3.35, 'avg_rss13': 17.5, 'var_rss13': 3.77, 'avg_rss23': 12.25, 'var_rss23': 3.9}

Streaming row 3 of 480:
{'# Columns: time': 500.0, 'avg_rss12': 35.5, 'var_rss12': 2.87, 'avg_rss13': 15.75, 'var_rss13': 2.86, 'avg_rss23': 17.75, 'var_rss23': 5.07}

Streaming row 4 of 480:
{'# Columns: time': 750.0, 'avg_rss12': 29.75, 'var_rss12': 12.19, 'avg_rss13': 16.25, 'var_rss13': 2.17, 'avg_rss23': 20.75, 'var_rss23': 2.59}

Streaming row 5 of 480:
{'# Columns: time': 1000.0, 'avg_rss12': 27.0, 'var_rss12': 2.12, 'avg_rss13': 16.75, 'var_rss13': 0.83, 'avg_rss23': 24.75, 'var_rss23': 1.64}

Streaming row 6 of 480:
{'# Columns: time': 1250.0, 'avg_rss12': 36.0, 'var_rss12': 5.61, 'avg_rss13': 12.25, 'var_rss13': 2.28, 'avg_r

In [ ]:
import pandas as pd


def sliding_window(data, window_size, stride):
    """
    Generate sliding windows from the dataset.
    Parameters:
        data (pd.DataFrame): The dataset.
        window_size (int): The size of each window.
        stride (int): The step size to move the window.
    Returns:
        List of sliding windows as DataFrames.
    """
    windows = []
    for i in range(0, len(data) - window_size + 1, stride):
        windows.append(data.iloc[i:i + window_size])
    return windows

def tumbling_window(data, window_size):
    """
    Generate tumbling windows from the dataset.
    Parameters:
        data (pd.DataFrame): The dataset.
        window_size (int): The size of each window.
    Returns:
        List of tumbling windows as DataFrames.
    """
    windows = []
    for i in range(0, len(data), window_size):
        windows.append(data.iloc[i:i + window_size])
    return windows

# Parameters
window_size = 5
stride = 2

# Extract sliding windows
sliding_windows = sliding_window(df, window_size, stride)
print(f"Sliding Windows (window size={window_size}, stride={stride}):")
for i, window in enumerate(sliding_windows):
    print(f"\nWindow {i + 1}:\n{window}")

# Extract tumbling windows
tumbling_windows = tumbling_window(df, window_size)
print(f"\nTumbling Windows (window size={window_size}):")
for i, window in enumerate(tumbling_windows):
    print(f"\nWindow {i + 1}:\n{window}")


In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Extract sliding windows (from the previous step)
def sliding_window(data, window_size, stride):
    windows = []
    for i in range(0, len(data) - window_size + 1, stride):
        windows.append(data.iloc[i:i + window_size])
    return windows

# Parameters
window_size = 5
stride = 2
sliding_windows = sliding_window(df, window_size, stride)

# Preprocess data (scaling for clustering)
scaler = StandardScaler()

# Perform clustering on each window
cluster_results = []
num_clusters = 3  # Number of clusters for k-means

for i, window in enumerate(sliding_windows):
    # Scale the data
    scaled_data = scaler.fit_transform(window)

    # Apply K-means
    kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
    kmeans.fit(scaled_data)

    # Store cluster labels and centroids
    cluster_results.append({
        "window_index": i,
        "labels": kmeans.labels_,
        "centroids": kmeans.cluster_centers_,
        "inertia": kmeans.inertia_  # Within-cluster sum of squares
    })

# Visualize clustering results
for result in cluster_results:
    print(f"Window {result['window_index'] + 1}:\nCluster Labels: {result['labels']}\n")
    print(f"Cluster Centroids:\n{result['centroids']}\n")
    print(f"Inertia (within-cluster sum of squares): {result['inertia']}\n")

# Visualize inertia (clustering performance across windows)
inertias = [result['inertia'] for result in cluster_results]
plt.plot(range(1, len(sliding_windows) + 1), inertias, marker='o')
plt.title("K-means Clustering Inertia Across Windows")
plt.xlabel("Window Index")
plt.ylabel("Inertia")
plt.show()


In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Scaling data
scaler = StandardScaler()

# Parameters
num_clusters = 3  # Number of clusters for k-means

# Perform clustering on each window
cluster_results = []

for i, window in enumerate(sliding_windows):
    # Scale the data
    scaled_data = scaler.fit_transform(window)

    # Apply K-means with explicit n_init
    kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
    kmeans.fit(scaled_data)

    # Store cluster results
    cluster_results.append({
        "window_index": i,
        "labels": kmeans.labels_,
        "centroids": kmeans.cluster_centers_,
        "inertia": kmeans.inertia_
    })

# Visualize inertia across windows
inertias = [result['inertia'] for result in cluster_results]
plt.plot(range(1, len(sliding_windows) + 1), inertias, marker='o')
plt.title("K-means Clustering Inertia Across Windows")
plt.xlabel("Window Index")
plt.ylabel("Inertia")
plt.show()


In [3]:
from sklearn.metrics import silhouette_score
import numpy as np

# Parameters
num_clusters = 3
silhouette_scores = []
inertias = []

# Perform clustering for different window sizes
for i, window in enumerate(sliding_windows):
    scaled_data = scaler.fit_transform(window)

    # Apply K-means
    kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
    kmeans.fit(scaled_data)

    # Calculate inertia and silhouette score
    inertias.append(kmeans.inertia_)
    if len(window) > 1:  # Silhouette score requires at least 2 data points per cluster
        score = silhouette_score(scaled_data, kmeans.labels_)
        silhouette_scores.append(score)
    else:
        silhouette_scores.append(np.nan)

# Plot inertia
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, len(sliding_windows) + 1), inertias, marker='o')
plt.title("Inertia Across Windows")
plt.xlabel("Window Index")
plt.ylabel("Inertia")

# Plot silhouette score
plt.subplot(1, 2, 2)
plt.plot(range(1, len(sliding_windows) + 1), silhouette_scores, marker='o', color='orange')
plt.title("Silhouette Score Across Windows")
plt.xlabel("Window Index")
plt.ylabel("Silhouette Score")

plt.tight_layout()
plt.show()


NameError: name 'sliding_windows' is not defined